In [1]:
# Importamos las librerías necesarias
from bs4 import BeautifulSoup
from selenium import webdriver
import selenium.webdriver.support.ui as UI
from selenium.webdriver.common.by import By
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
import contextlib
import pandas as pd
import requests
import time

In [2]:
# Cargamos el archivo txt que contiene el PATH del driver y lo
# guardamos en una variable
with open("path driver firefox.txt", encoding="utf-8") as file:
    for line in file:
        PATH = line

In [3]:
# Preparamos la página que usaremos para el raspado de web, mediante el
# uso de Selenium

# Establecemos el driver (en este caso con Firefox)

driver = webdriver.Firefox(executable_path=PATH)

# Lo que haremos será clickar el boton de acepar cookies y luego el de "Show More"
with contextlib.closing(driver) as browser:
    # Obtenemos la página
    browser.get(
        'https://www.espn.com/nba/stats/player/_/table/offensive/sort/avgPoints/dir/desc')
    # Maximizamos la pantalla
    driver.maximize_window()
    # Esperamos hasta que aparezca la ventana de las cookies y las aceptamos
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((
        By.XPATH,'//*[@id="onetrust-accept-btn-handler"]'))).click()
    # Esperamos unos segundos a que se recargue la página
    wait = UI.WebDriverWait(browser, 10)
    # Guardamos la longitud del número de clicks que tiene el botón
    find_length = len(driver.find_elements_by_xpath(
        '/html/body/div[1]/div/div/div/div[4]/div[3]/div/div/section/div/div[3]/div[2]/a'))
    # Establecemos a 0 una variable que nos servirá para el bucle
    list_count = 0
    # Clickamos en el botón 'Show More' tantas veces mientras la longitud del
    # número de clicks sea mayor que 'list_count'
    try:
        while (list_count < find_length):    
            next_page_link = wait.until(
                EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, 'Show More')))
            next_page_link.click()
    # Si hay una excepción del tipo TimeoutException, guardar la fuente de la 
    # página en una variable
    except TimeoutException:
        page_source = driver.page_source
        
    

# Web Scraping

# Una vez tenemos la página como nos interesa, realizamos el raspado de web

# Establecemos los headers
headers = {
    "user-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"
}

# Creamos la 'sopa' utilizando la página que habíamos obtenido previamente
soup = BeautifulSoup(page_source, "html5lib")

nba_table_one = soup.find('div', attrs={'class': "flex"})

for div in nba_table_one:
    # Ranking y Nombre de los jugadores
    nba_players = soup.find('table',
                            attrs={'class': "Table Table--align-right Table--fixed Table--fixed-left"})
    # Lista vacía que contendrá la cabecera de las variables
    cabecera = []
    # Recogemos la cabecera de la página
    for tr in nba_players.find_all('tr'):
        for th in tr.find_all('th'):
            cabecera.append(th.get_text())
    # Lista vacía que contendrá los datos
    data = []
    # Recogemos los datos de la página
    for tr in nba_players.find_all('tr'):
        t_row = {}
        for td, th in zip(tr.find_all('td'), cabecera):
            t_row[th] = td.text.replace('\n', '').strip()
        data.append(t_row)
            
    nba_table = soup.find('table', attrs={'class': "Table Table--align-right"})
    # Lista vacía que contendrá la cabecera de las variables
    cabecera_2 = []
    # Recogemos la cabecera de la página
    for tr in nba_table.find_all('tr'):
        for th in tr.find_all('th'):
            cabecera_2.append(th.get_text())
    # Lista vacía que contendrá los datos
    data_2 = []
    # Recogemos los datos de la página
    for tr in nba_table.find_all('tr'):
        t_row = {}
        for td, th in zip(tr.find_all('td'), cabecera_2):
            t_row[th] = td.text.replace('\n', '').strip()
        data_2.append(t_row)


# Establecemos dos dataframe borrando las posibles fials con valores nulos
df_names = pd.DataFrame(data).dropna()
df_stats = pd.DataFrame(data_2).dropna()

# Unimos los dos dataframes y establecemos un índice:
nba_dataframe = df_names.join(df_stats).set_index('RK')

# Convertimos el dataframe en un CSV:
nba_dataframe.to_csv("NBA Stats.csv", index=True)

# Mostramos el dataframe por pantalla
nba_dataframe

,Name,POS,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,...,FTA,FT%,REB,AST,STL,BLK,TO,DD2,TD3,PER
RK,,,,,,,,,,,,,,,,,,,,,
1,Bradley BealWSH,SG,39,35.7,31.8,11.2,22.9,48.8,2.2,6.5,...,8.1,90.2,5.1,4.6,1.3,0.4,3.3,3,0,23.82
2,Damian LillardPOR,PG,42,36.0,30.1,9.3,20.6,45.2,4.3,11.3,...,7.7,94.1,4.4,7.7,0.9,0.2,3.3,10,0,26.91
3,Joel EmbiidPHI,C,31,32.6,29.9,9.5,18.0,52.5,1.2,2.9,...,11.4,85.9,11.5,3.3,1.2,1.4,3.2,21,0,31.63
4,Giannis AntetokounmpoMIL,PF,40,34.2,29.0,10.6,18.8,56.2,1.2,3.9,...,10.0,67.9,11.7,6.4,1.2,1.3,3.7,30,7,29.31
5,Stephen CurryGS,PG,39,33.7,29.0,9.6,20.2,47.5,4.7,11.4,...,5.5,93.0,5.3,6.2,1.4,0.1,3.1,7,0,24.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,Matisse ThybullePHI,SG,41,18.7,3.7,1.4,3.4,41.1,0.7,2.1,...,0.3,57.1,1.7,1.0,1.5,1.0,0.4,0,0,10.26
247,Gary ClarkORL,SF,35,18.2,3.4,1.1,3.7,30.5,0.9,3.1,...,0.3,80.0,3.2,0.9,0.3,0.2,0.5,0,0,6.10
248,Stanley JohnsonTOR,SF,38,14.2,2.9,1.0,2.7,37.6,0.6,1.7,...,0.4,78.6,2.2,1.2,0.7,0.3,0.7,0,0,7.83


In [4]:
# Vamos ahora a cargar el archivo CSV para comprobar que todo está bien
nba = pd.read_csv('NBA Stats.csv').set_index('RK')
nba

,Name,POS,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,...,FTA,FT%,REB,AST,STL,BLK,TO,DD2,TD3,PER
RK,,,,,,,,,,,,,,,,,,,,,
1,Bradley BealWSH,SG,39,35.7,31.8,11.2,22.9,48.8,2.2,6.5,...,8.1,90.2,5.1,4.6,1.3,0.4,3.3,3,0,23.82
2,Damian LillardPOR,PG,42,36.0,30.1,9.3,20.6,45.2,4.3,11.3,...,7.7,94.1,4.4,7.7,0.9,0.2,3.3,10,0,26.91
3,Joel EmbiidPHI,C,31,32.6,29.9,9.5,18.0,52.5,1.2,2.9,...,11.4,85.9,11.5,3.3,1.2,1.4,3.2,21,0,31.63
4,Giannis AntetokounmpoMIL,PF,40,34.2,29.0,10.6,18.8,56.2,1.2,3.9,...,10.0,67.9,11.7,6.4,1.2,1.3,3.7,30,7,29.31
5,Stephen CurryGS,PG,39,33.7,29.0,9.6,20.2,47.5,4.7,11.4,...,5.5,93.0,5.3,6.2,1.4,0.1,3.1,7,0,24.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,Matisse ThybullePHI,SG,41,18.7,3.7,1.4,3.4,41.1,0.7,2.1,...,0.3,57.1,1.7,1.0,1.5,1.0,0.4,0,0,10.26
247,Gary ClarkORL,SF,35,18.2,3.4,1.1,3.7,30.5,0.9,3.1,...,0.3,80.0,3.2,0.9,0.3,0.2,0.5,0,0,6.10
248,Stanley JohnsonTOR,SF,38,14.2,2.9,1.0,2.7,37.6,0.6,1.7,...,0.4,78.6,2.2,1.2,0.7,0.3,0.7,0,0,7.83
